In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Point
from shapely.geometry import Polygon

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [2]:
pwd

u'/Users/kristikorsberg/devel/big_data/big_data_prj/Analyzing_Noise_in_NYC/py'

In [3]:
ct = gpd.read_file('../data/nyct2010_17a/nyct2010.shp')

In [4]:
ct.head(2)

,BoroCT2010,BoroCode,BoroName,CDEligibil,CT2010,CTLabel,NTACode,NTAName,PUMA,Shape_Area,Shape_Leng,geometry
0,5000900,5,Staten Island,I,000900,9,SI22,West New Brighton-New Brighton-St. George,3903,2.497010e+06,7729.016794,"POLYGON ((962269.1260375977 173705.5001831055,..."
1,1009800,1,Manhattan,I,009800,98,MN19,Turtle Bay-East Midtown,3808,1.906016e+06,5534.199811,"POLYGON ((994133.507019043 214848.8975830078, ..."


In [5]:
noise = pd.read_csv('/Users/kristikorsberg/Downloads/311_Service_Requests_from_2010_to_Present.csv')

//anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (8,17,40,43,44,45,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
noise.head(2)

,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,...,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Garage Lot Name,Ferry Direction,Ferry Terminal Name,Latitude,Longitude,Location
0,35607069,03/02/2017 03:33:17 PM,03/12/2017 11:21:04 AM,HPD,Department of Housing Preservation and Develop...,FLOORING/STAIRS,FLOOR,RESIDENTIAL BUILDING,11220,6123 4 AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.640087,-74.019194,"(40.6400872732656, -74.01919432059279)"
1,35597174,03/01/2017 05:10:34 PM,03/12/2017 09:40:02 AM,HPD,Department of Housing Preservation and Develop...,DOOR/WINDOW,DOOR,RESIDENTIAL BUILDING,10467,3301 PERRY AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.877016,-73.875373,"(40.877016358193174, -73.87537267430177)"


In [7]:
noise.columns

Index([u'Unique Key', u'Created Date', u'Closed Date', u'Agency',
       u'Agency Name', u'Complaint Type', u'Descriptor', u'Location Type',
       u'Incident Zip', u'Incident Address', u'Street Name', u'Cross Street 1',
       u'Cross Street 2', u'Intersection Street 1', u'Intersection Street 2',
       u'Address Type', u'City', u'Landmark', u'Facility Type', u'Status',
       u'Due Date', u'Resolution Description',
       u'Resolution Action Updated Date', u'Community Board', u'Borough',
       u'X Coordinate (State Plane)', u'Y Coordinate (State Plane)',
       u'Park Facility Name', u'Park Borough', u'School Name',
       u'School Number', u'School Region', u'School Code',
       u'School Phone Number', u'School Address', u'School City',
       u'School State', u'School Zip', u'School Not Found',
       u'School or Citywide Complaint', u'Vehicle Type',
       u'Taxi Company Borough', u'Taxi Pick Up Location',
       u'Bridge Highway Name', u'Bridge Highway Direction', u'Road Ramp',

In [8]:
noise.head(2)

,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,...,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Garage Lot Name,Ferry Direction,Ferry Terminal Name,Latitude,Longitude,Location
0,35607069,03/02/2017 03:33:17 PM,03/12/2017 11:21:04 AM,HPD,Department of Housing Preservation and Develop...,FLOORING/STAIRS,FLOOR,RESIDENTIAL BUILDING,11220,6123 4 AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.640087,-74.019194,"(40.6400872732656, -74.01919432059279)"
1,35597174,03/01/2017 05:10:34 PM,03/12/2017 09:40:02 AM,HPD,Department of Housing Preservation and Develop...,DOOR/WINDOW,DOOR,RESIDENTIAL BUILDING,10467,3301 PERRY AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.877016,-73.875373,"(40.877016358193174, -73.87537267430177)"


In [9]:
noise = noise[['Unique Key', 'Created Date', 'Closed Date', 'Agency', 'Complaint Type', 'Descriptor', 'Incident Zip', \
               'X Coordinate (State Plane)', 'Y Coordinate (State Plane)']]

In [10]:
noise = noise.dropna()

In [11]:
geometry = [Point(xy) for xy in zip(noise['X Coordinate (State Plane)'], noise['Y Coordinate (State Plane)'])]
noise = noise.drop(['X Coordinate (State Plane)', 'Y Coordinate (State Plane)'], axis=1)
crs = {'init': 'epsg:2263'}
geo_noise = gpd.GeoDataFrame(noise, crs=crs, geometry=geometry)

In [12]:
print geo_noise.crs
print ct.crs

{'init': 'epsg:2263'}
{u'lon_0': -74, u'datum': u'NAD83', u'y_0': 0, u'no_defs': True, u'proj': u'lcc', u'x_0': 300000, u'units': u'us-ft', u'lat_2': 41.03333333333333, u'lat_1': 40.66666666666666, u'lat_0': 40.16666666666666}


In [13]:
ct = ct.to_crs({'init':'epsg:2263'})

In [14]:
print geo_noise.crs
print ct.crs

{'init': 'epsg:2263'}
{'init': 'epsg:2263'}


In [15]:
ct = ct[['geometry', 'CT2010', 'Shape_Area', 'Shape_Leng']]

In [16]:
ct.columns

Index([u'geometry', u'CT2010', u'Shape_Area', u'Shape_Leng'], dtype='object')

In [17]:
noise_simple = geo_noise[['Unique Key', 'geometry']]

In [18]:
#http://geoffboeing.com/2016/10/r-tree-spatial-index-python/
precise_matches = {}
spatial_index = geo_noise.sindex
for i, CT2010 in enumerate(ct.CT2010):
    possible_matches_index = list(spatial_index.intersection(ct['geometry'][i].bounds))
    possible_matches = geo_noise.iloc[possible_matches_index]
    precise_matches[CT2010] = possible_matches[possible_matches.intersects(ct['geometry'][i])]
    #precise_matches[CT2010]['CT2010'] = CT2010 

In [19]:
noise_ct = pd.concat(precise_matches).reset_index()

In [20]:
noise_ct.head(2)

,level_0,level_1,Agency,Closed Date,Complaint Type,Created Date,Descriptor,Incident Zip,Unique Key,geometry
0,000100,203592,DOB,04/10/2017 02:02:26 PM,Special Enforcement,04/10/2017 02:02:26 PM,SEP - Professional Certification Compliance Audit,11201,35909960.0,POINT (986020 194280)
1,000100,178552,DEP,03/29/2017 05:00:00 PM,Lead,03/28/2017 05:00:00 PM,Lead Kit Request (Residential) (L10),11201,35812477.0,POINT (986751 194303)


In [21]:
noise_ct = noise_ct.rename(columns={'level_0':'CT2010'})
noise_ct = noise_ct.drop('level_1',axis=1)

In [22]:
noise_ct.head(2)

,CT2010,Agency,Closed Date,Complaint Type,Created Date,Descriptor,Incident Zip,Unique Key,geometry
0,000100,DOB,04/10/2017 02:02:26 PM,Special Enforcement,04/10/2017 02:02:26 PM,SEP - Professional Certification Compliance Audit,11201,35909960.0,POINT (986020 194280)
1,000100,DEP,03/29/2017 05:00:00 PM,Lead,03/28/2017 05:00:00 PM,Lead Kit Request (Residential) (L10),11201,35812477.0,POINT (986751 194303)


In [24]:
noise_ct['CT2010'] = noise_ct['CT2010'].astype(str)

In [25]:
noise_ct['Census Tract'] = noise_ct['CT2010'].apply(lambda s: s.lstrip("0"))

In [28]:
noise_ct.head(2)

,CT2010,Agency,Closed Date,Complaint Type,Created Date,Descriptor,Incident Zip,Unique Key,geometry,Census Tract
0,000100,DOB,04/10/2017 02:02:26 PM,Special Enforcement,04/10/2017 02:02:26 PM,SEP - Professional Certification Compliance Audit,11201,35909960.0,POINT (986020 194280),100
1,000100,DEP,03/29/2017 05:00:00 PM,Lead,03/28/2017 05:00:00 PM,Lead Kit Request (Residential) (L10),11201,35812477.0,POINT (986751 194303),100


In [26]:
pop = pd.read_csv('../data/New_York_City_Population_By_Census_Tracts.csv')

In [27]:
pop.head(2)

,Borough,Year,FIPS County Code,DCP Borough Code,Census Tract,Population
0,Bronx,2000,5,2,100,12780
1,Bronx,2000,5,2,200,3545


In [29]:
pop = pop[pop['Year']==2010]

In [30]:
pop['Census Tract'] = pop['Census Tract'].astype(str)
noise_ct['Census Tract'] = noise_ct['Census Tract'].astype(str)

In [31]:
pop.head(2)

,Borough,Year,FIPS County Code,DCP Borough Code,Census Tract,Population
2168,Bronx,2010,5,2,100,11091
2169,Bronx,2010,5,2,200,4334


In [32]:
ct_noise_pop = pd.merge(noise_ct, pop, on='Census Tract', how='outer')

In [33]:
ct_noise_pop.head(2)

,CT2010,Agency,Closed Date,Complaint Type,Created Date,Descriptor,Incident Zip,Unique Key,geometry,Census Tract,Borough,Year,FIPS County Code,DCP Borough Code,Population
0,000100,DOB,04/10/2017 02:02:26 PM,Special Enforcement,04/10/2017 02:02:26 PM,SEP - Professional Certification Compliance Audit,11201,35909960.0,POINT (986020 194280),100,Bronx,2010,5,2,11091
1,000100,DOB,04/10/2017 02:02:26 PM,Special Enforcement,04/10/2017 02:02:26 PM,SEP - Professional Certification Compliance Audit,11201,35909960.0,POINT (986020 194280),100,Brooklyn,2010,47,3,4338


In [34]:
type(ct_noise_pop)

geopandas.geodataframe.GeoDataFrame

In [38]:
print len(ct_noise_pop['Unique Key'].unique())
print len(noise['Unique Key'].unique())
print len(noise_ct['Unique Key'].unique())

135487
224925
135486


In [36]:
#ct_noise_pop.to_file('../data/ct_noise_pop/ct_noise_pop.shp',driver='ESRI Shapefile')